# Azure File Transaltion

## Overview

### Integration details

### Tool features

The AzureFileTranslateTool is a wrapper around the Azure Text Translation API that translates text from documents in various formats to a target language.

This tool supports:

PDF, DOCX, PPTX (Powerpoint), XLSX (Excel), XML, HTML, and TXT files.

## Setup

This section provides details about how the Azure AI Face integration works, including setup.

In [ ]:
import os

from langchain import hub


We will use a prompt to instruct the model. LangChain prompts can be configured, but for simplicity, we will use a premade prompt from LangSmith. This requires an API key, which can be set up here after registration.

In [ ]:
LANGSMITH_KEY = os.environ["LANGSMITH_KEY"]
prompt = hub.pull("hwchase17/structured-chat-agent", api_key=LANGSMITH_KEY)

Now we can use the AzureFileTranslateTool combined with a model, using create_structured_chat_agent.

In [ ]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.azure_ai_services.azure_file_translation import (
    AzureFileTranslateTool,
)
from langchain_openai import AzureChatOpenAI

### Credentials

Credentials can be set by being passed as parameters and should be stored locally as environment variables.

In [ ]:
azure_translate_key = os.environ["AZURE_TRANSLATE_API_KEY"]
azure_translate_endpoint = os.environ["AZURE_TRANSLATE_ENDPOINT"]
region = os.environ["AZURE_REGION"]

## Instantiation

Credentials can be passed directly, but they can also be retrieved automatically by the constructor if environment variables named AZURE_TRANSLATE_ENDPOINT, AZURE_TRANSLATE_API_KEY, and region are set.

In [ ]:
ft = AzureFileTranslateTool(
    text_translation_key=os.environ["AZURE_TRANSLATE_API_KEY"],
    text_translation_endpoint=os.environ["AZURE_TRANSLATE_ENDPOINT"],
    region="westus2",
)

In [ ]:
tools = [ft]

model = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_deployment=os.environ["COMPLETIONS_MODEL"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

Creating an AgentExecutor chain allows a model to use tools to assist in it's response.

In [ ]:
agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

## Invocation

Get an input file path

In [ ]:
from pathlib import Path

_THIS_DIR = Path(__file__).parents[2]
_EXAMPLE_DIR = _THIS_DIR / "examples"
AZURE_PDF = _EXAMPLE_DIR / "test_azure.pdf"

file_path: str = AZURE_PDF

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

Firstly, the tool can be used by directly passing input as an argument. However, this is discouraged as the tool is intended to be used in an executor chain.

In [ ]:
ft._run(input)

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

By using .invoke, the model can be told what to do and assess if using the tools it was given would assist in it's response. This is the intended use.

In [ ]:
answer = agent_executor.invoke(
    {"input": f"What does the text in the file say in english? : {file_path}"}
)

## Chaining

When creating an AgentExecutor as described earlier, an execution chain is formed. The sequence of events will be printed using the given prompt, and actions will occur in a chain-like manner.

In [ ]:
agent = create_structured_chat_agent(model, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

## API reference 
[Azure Text Translation Overview](https://learn.microsoft.com/en-us/azure/ai-services/translator/text-translation-overview)